# Reti generative

Le Reti Neurali Ricorrenti (RNN) e le loro varianti con celle gated, come le celle Long Short Term Memory (LSTM) e le Gated Recurrent Units (GRU), hanno fornito un meccanismo per il modellamento del linguaggio, ovvero possono apprendere l'ordine delle parole e fornire previsioni per la parola successiva in una sequenza. Questo ci permette di utilizzare le RNN per **compiti generativi**, come la generazione di testo ordinario, la traduzione automatica e persino la descrizione di immagini.

Nell'architettura RNN che abbiamo discusso nell'unità precedente, ogni unità RNN produceva il prossimo stato nascosto come output. Tuttavia, possiamo anche aggiungere un altro output a ciascuna unità ricorrente, che ci consentirebbe di generare una **sequenza** (di lunghezza uguale alla sequenza originale). Inoltre, possiamo utilizzare unità RNN che non accettano un input a ogni passo, ma prendono solo un vettore di stato iniziale e poi producono una sequenza di output.

In questo notebook, ci concentreremo su modelli generativi semplici che ci aiutano a generare testo. Per semplicità, costruiamo una **rete a livello di carattere**, che genera testo lettera per lettera. Durante l'addestramento, dobbiamo prendere un corpus di testo e suddividerlo in sequenze di lettere.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Costruire un vocabolario di caratteri

Per costruire una rete generativa a livello di caratteri, dobbiamo suddividere il testo in singoli caratteri invece che in parole. Il livello `TextVectorization` che abbiamo utilizzato in precedenza non può farlo, quindi abbiamo due opzioni:

* Caricare manualmente il testo e fare la tokenizzazione "a mano", come mostrato in [questo esempio ufficiale di Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Utilizzare la classe `Tokenizer` per la tokenizzazione a livello di caratteri.

Opteremo per la seconda opzione. `Tokenizer` può essere utilizzato anche per la tokenizzazione in parole, quindi dovrebbe essere possibile passare facilmente dalla tokenizzazione a livello di caratteri a quella a livello di parole.

Per effettuare la tokenizzazione a livello di caratteri, dobbiamo passare il parametro `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Vogliamo anche utilizzare un token speciale per indicare **fine della sequenza**, che chiameremo `<eos>`. Aggiungiamolo manualmente al vocabolario:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Addestrare un RNN generativo per generare titoli

Il modo in cui addestreremo l'RNN per generare titoli di notizie è il seguente. A ogni passo, prenderemo un titolo, che verrà fornito a un RNN, e per ogni carattere di input chiederemo alla rete di generare il carattere di output successivo:

![Immagine che mostra un esempio di generazione RNN della parola 'HELLO'.](../../../../../translated_images/it/rnn-generate.56c54afb52f9781d.webp)

Per l'ultimo carattere della nostra sequenza, chiederemo alla rete di generare il token `<eos>`.

La principale differenza con l'RNN generativo che stiamo utilizzando qui è che prenderemo un output da ogni passo dell'RNN, e non solo dalla cella finale. Questo può essere ottenuto specificando il parametro `return_sequences` alla cella RNN.

Quindi, durante l'addestramento, un input per la rete sarà una sequenza di caratteri codificati di una certa lunghezza, e un output sarà una sequenza della stessa lunghezza, ma spostata di un elemento e terminata con `<eos>`. Il minibatch sarà composto da diverse di queste sequenze, e sarà necessario utilizzare **padding** per allineare tutte le sequenze.

Creiamo funzioni che trasformeranno il dataset per noi. Poiché vogliamo aggiungere padding alle sequenze a livello di minibatch, prima raggrupperemo il dataset chiamando `.batch()`, e poi lo `map` per effettuare la trasformazione. Quindi, la funzione di trasformazione prenderà un intero minibatch come parametro:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Alcune cose importanti che facciamo qui:
* Per prima cosa estraiamo il testo effettivo dal tensore di stringhe
* `text_to_sequences` converte l'elenco di stringhe in un elenco di tensori interi
* `pad_sequences` riempie quei tensori fino alla loro lunghezza massima
* Infine, codifichiamo tutti i caratteri in one-hot, e facciamo anche lo spostamento e l'aggiunta di `<eos>`. Presto vedremo perché abbiamo bisogno di caratteri codificati in one-hot.

Tuttavia, questa funzione è **Pythonic**, ovvero non può essere tradotta automaticamente nel grafo computazionale di Tensorflow. Otterremo errori se proviamo a utilizzare questa funzione direttamente nella funzione `Dataset.map`. Dobbiamo racchiudere questa chiamata Pythonic utilizzando il wrapper `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Nota**: Differenziare tra funzioni di trasformazione Pythonic e Tensorflow potrebbe sembrare un po' troppo complesso, e potresti chiederti perché non trasformiamo il dataset utilizzando funzioni Python standard prima di passarlo a `fit`. Sebbene ciò sia sicuramente possibile, utilizzare `Dataset.map` ha un enorme vantaggio, poiché la pipeline di trasformazione dei dati viene eseguita utilizzando il grafo computazionale di Tensorflow, che sfrutta i calcoli della GPU e riduce al minimo la necessità di trasferire dati tra CPU e GPU.

Ora possiamo costruire la nostra rete generativa e iniziare l'addestramento. Può essere basata su qualsiasi cella ricorrente di cui abbiamo discusso nell'unità precedente (semplice, LSTM o GRU). Nel nostro esempio utilizzeremo LSTM.

Poiché la rete prende caratteri come input e la dimensione del vocabolario è piuttosto piccola, non abbiamo bisogno di uno strato di embedding: l'input codificato in one-hot può essere passato direttamente alla cella LSTM. Lo strato di output sarà un classificatore `Dense` che convertirà l'output dell'LSTM in numeri di token codificati in one-hot.

Inoltre, poiché stiamo lavorando con sequenze di lunghezza variabile, possiamo utilizzare uno strato `Masking` per creare una maschera che ignorerà la parte riempita della stringa. Questo non è strettamente necessario, poiché non siamo particolarmente interessati a tutto ciò che va oltre il token `<eos>`, ma lo utilizzeremo per acquisire un po' di esperienza con questo tipo di strato. L'`input_shape` sarà `(None, vocab_size)`, dove `None` indica la sequenza di lunghezza variabile, e la forma dell'output sarà anch'essa `(None, vocab_size)`, come puoi vedere dal `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Generazione dell'output

Ora che abbiamo addestrato il modello, vogliamo usarlo per generare un output. Prima di tutto, abbiamo bisogno di un modo per decodificare il testo rappresentato da una sequenza di numeri di token. Per fare questo, potremmo utilizzare la funzione `tokenizer.sequences_to_texts`; tuttavia, non funziona bene con la tokenizzazione a livello di carattere. Pertanto, prenderemo un dizionario di token dal tokenizer (chiamato `word_index`), costruiremo una mappa inversa e scriveremo la nostra funzione di decodifica:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Ora iniziamo la generazione. Partiremo da una stringa `start`, la codificheremo in una sequenza `inp`, e ad ogni passo chiameremo la nostra rete per inferire il carattere successivo.

L'output della rete `out` è un vettore di `vocab_size` elementi che rappresentano le probabilità di ciascun token, e possiamo trovare il numero del token più probabile utilizzando `argmax`. Successivamente, aggiungiamo questo carattere alla lista generata di token e procediamo con la generazione. Questo processo di generazione di un carattere viene ripetuto `size` volte per generare il numero richiesto di caratteri, e terminiamo anticipatamente quando viene incontrato `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Campionamento dell'output durante l'addestramento

Poiché non abbiamo metriche utili come *accuratezza*, l'unico modo per verificare che il nostro modello stia migliorando è **campionare** le stringhe generate durante l'addestramento. Per farlo, utilizzeremo **callback**, ovvero funzioni che possiamo passare alla funzione `fit` e che verranno chiamate periodicamente durante l'addestramento.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Questo esempio genera già un testo piuttosto buono, ma può essere ulteriormente migliorato in diversi modi:

* **Più testo**. Abbiamo utilizzato solo titoli per il nostro compito, ma potresti voler sperimentare con il testo completo. Ricorda che gli RNN non sono molto efficaci nel gestire sequenze lunghe, quindi ha senso suddividerle in frasi più brevi o allenarsi sempre su una lunghezza di sequenza fissa di un valore predefinito `num_chars` (ad esempio, 256). Potresti provare a modificare l'esempio sopra in un'architettura simile, utilizzando il [tutorial ufficiale di Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) come ispirazione.

* **LSTM multilivello**. Ha senso provare 2 o 3 livelli di celle LSTM. Come abbiamo menzionato nell'unità precedente, ogni livello di LSTM estrae determinati schemi dal testo e, nel caso di un generatore a livello di carattere, possiamo aspettarci che il livello inferiore di LSTM sia responsabile dell'estrazione delle sillabe, mentre i livelli superiori si occupano di parole e combinazioni di parole. Questo può essere implementato semplicemente passando un parametro relativo al numero di livelli al costruttore di LSTM.

* Potresti anche voler sperimentare con **unità GRU** per vedere quali offrono prestazioni migliori e con **dimensioni diverse dei livelli nascosti**. Un livello nascosto troppo grande potrebbe portare a overfitting (ad esempio, la rete apprenderà il testo esatto), mentre una dimensione più piccola potrebbe non produrre risultati soddisfacenti.


## Generazione di testo morbido e temperatura

Nella precedente definizione di `generate`, sceglievamo sempre il carattere con la probabilità più alta come prossimo carattere nel testo generato. Questo portava spesso al fatto che il testo "ciclasse" tra le stesse sequenze di caratteri più e più volte, come in questo esempio:
```
today of the second the company and a second the company ...
```

Tuttavia, se osserviamo la distribuzione di probabilità per il prossimo carattere, potrebbe accadere che la differenza tra alcune delle probabilità più alte non sia così grande, ad esempio un carattere potrebbe avere una probabilità di 0,2, un altro di 0,19, ecc. Per esempio, quando cerchiamo il prossimo carattere nella sequenza '*play*', il carattere successivo potrebbe essere ugualmente uno spazio o **e** (come nella parola *player*).

Questo ci porta alla conclusione che non è sempre "giusto" selezionare il carattere con la probabilità più alta, perché scegliere il secondo più probabile potrebbe comunque portarci a un testo significativo. È più saggio **campionare** i caratteri dalla distribuzione di probabilità fornita dall'output della rete.

Questo campionamento può essere effettuato utilizzando la funzione `np.multinomial`, che implementa la cosiddetta **distribuzione multinomiale**. Una funzione che implementa questa generazione di testo **morbida** è definita di seguito:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Abbiamo introdotto un altro parametro chiamato **temperatura**, che viene utilizzato per indicare quanto strettamente dobbiamo aderire alla probabilità più alta. Se la temperatura è 1.0, facciamo un campionamento multinomiale equo, e quando la temperatura va all'infinito - tutte le probabilità diventano uguali, e selezioniamo casualmente il prossimo carattere. Nell'esempio qui sotto possiamo osservare che il testo diventa privo di senso quando aumentiamo troppo la temperatura, e somiglia a un testo "ciclato" generato rigidamente quando si avvicina a 0.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall'uso di questa traduzione.
